In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_16_FT_WritingPrompts_full.xlsx'
df = pd.read_excel(file_path, sheet_name ='Sheet1', engine = 'openpyxl')

In [ ]:
# Load the model
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Define columns for prompts and responses
prompts = df['Prompts']
Functional = df['Functionality']
response_1 = df['Response_1']
response_2 = df['Response_2']
response_3 = df['Response_3']

In [ ]:
# Initialize a list to store average similarity scores and detect functionality of responses
average_similarities = []
# Iterate through each prompt and its three responses
for prompt, func, resp1, resp2, resp3 in zip(prompts, Functional ,response_1, response_2, response_3):
    # Get embeddings for prompt and each response
    embedding_prompt = model.encode(prompt)
    resp1 = str(resp1)
    resp2 = str(resp2)
    resp3 = str(resp3)
    embedding_resp1 = model.encode(resp1)
    embedding_resp2 = model.encode(resp2)
    embedding_resp3 = model.encode(resp3)

    # Normalize embeddings for better cosine similarity measurement
    embedding_prompt_normalized = normalize(embedding_prompt.reshape(1, -1))
    embedding_resp1_normalized = normalize(embedding_resp1.reshape(1, -1))
    embedding_resp2_normalized = normalize(embedding_resp2.reshape(1, -1))
    embedding_resp3_normalized = normalize(embedding_resp3.reshape(1, -1))

    # Calculate cosine similarity between prompt and each response
    similarity_1 = cosine_similarity(embedding_prompt_normalized, embedding_resp1_normalized)[0][0]
    similarity_2 = cosine_similarity(embedding_prompt_normalized, embedding_resp2_normalized)[0][0]
    similarity_3 = cosine_similarity(embedding_prompt_normalized, embedding_resp3_normalized)[0][0]

    # Calculate the average similarity and functionality
    average_similarity = np.mean([similarity_1, similarity_2, similarity_3])
    average_similarities.append((float(func)/3)*average_similarity)

In [ ]:
df['Fluency'] = average_similarities

In [ ]:
# Save the updated DataFrame back to Google Drive
updated_file_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_16_FT_WritingPrompts_full.xlsx'
df.to_excel(updated_file_path, index=False, engine='openpyxl')